In [2]:
import json
with open('C:/Users/dvive/OneDrive/Desktop/reactapp/Project/Data/config.json', "r") as json_file:
    config_file = json_file.read()

config_file = json.loads(config_file) 

primary_key = "sql_steps"

print("Type of Config File : ",type(config_file))
print("Keys in Config File : ",config_file.keys())
print("The Values of sql_steps is a list: ",isinstance(config_file["sql_steps"],list))
print("Value of Key 'sql_steps' : ",config_file['sql_steps'])

config_file.setdefault(primary_key,"Key Not found") 

def validate_datatype(var,datatype):
    if type(var) == datatype:
        return f"Datatype {datatype} Matched"
    else:
        return f"Datatype {datatype} not Matched"
    
print(validate_datatype(config_file,dict))

def check_keys(*args):
    for i in range(len(config_file[primary_key])):
        if config_file[primary_key][i].get(args[0]) is not None and config_file[primary_key][i].get(args[1]) is not None and config_file[primary_key][i].get(args[2]):
            print("Keys Found in Block: ",i)
        else:
            print("Keys not Found in Block: ",i)
    
check_keys("sql_num","temp_view","next_sql")    

Type of Config File :  <class 'dict'>
Keys in Config File :  dict_keys(['sql_steps'])
The Values of sql_steps is a list:  True
Value of Key 'sql_steps' :  [{'sql_num': '1', 'action': 'read|publish', 'sql': "select original_claim_num_cd, parent_claim_num_cd, child_claim_num_cd, claim_ordinal_value_num from ods_vw_service.mv_consume_claim_ln_reprocess_assn where claim_ln_reprocess_tp_cd = 'RORGCL' union select original_claim_num_cd, parent_claim_num_cd, child_claim_num_cd, claim_ordinal_value_num from ods_vw_service.mv_consume_claim_ln_reprocess_assn where claim_ln_reprocess_tp_cd != 'RORGCL' and parent_claim_num_cd != child_claim_num_cd", 'temp_view': 'consume_claim_ln_reprocess_assn', 'next_sql': '2'}, {'sql_num': '2', 'action': 'write|parquet', 'sql': 'select * from consume_claim_ln_reprocess_assn', 'location': 'sandbox/claims_reprocess/06-07/claim_ln_repr_ordinal_2/', 'mode': 'overwrite', 'next_sql': '3'}, {'sql_num': '3', 'action': 'read|parquet', 'location': 'sandbox/claims_reproce